# ETL 

#### Importamos las librerias

In [2]:
import pandas as pd
import numpy as np
import re

#### Cargamos los archivos .csv

In [3]:
movies = pd.read_csv("C:/Users/JESHUDELL/Desktop/Proyecto Individual jesu/Dataset/1movies_dataset.csv")
credits = pd.read_csv("C:/Users/JESHUDELL/Desktop/Proyecto Individual jesu/Dataset/1credits.csv")

C:\Users\JESHUDELL\AppData\Local\Temp\ipykernel_6128\3109168342.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("C:/Users/JESHUDELL/Desktop/Proyecto Individual jesu/Dataset/1movies_dataset.csv")


In [4]:
# Verificamos cabtidad de registros por cada una
print(movies.shape, credits.shape)

(45466, 24) (45476, 3)


#### Visualización de los Datasets

In [5]:
# Visualizacion de registros de cada Dataset
display(movies.head(), credits.head())

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [6]:
# Revisamos el timpo de dato de los valores de cada Dataset
movies.info()
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

#### Transformacion de Datos

In [7]:
# Función para extraer y concatenar datos
def extract_concatenate_data(pattern, text):
    matches = re.findall(pattern, text)
    return ', '.join(matches)

# Definir los patrones de extracción
name_pattern = r"'name':\s+'(.*?)'"
director_pattern = r"'job': 'Director', 'name':\s+'(.*?)'"

# Normalizar las columnas en movies
movies['collection'] = movies['belongs_to_collection'].apply(lambda x: extract_concatenate_data(name_pattern, str(x)))
movies['genre'] = movies['genres'].apply(lambda x: extract_concatenate_data(name_pattern, str(x)))
movies['company'] = movies['production_companies'].apply(lambda x: extract_concatenate_data(name_pattern, str(x)))
movies['country'] = movies['production_countries'].apply(lambda x: extract_concatenate_data(name_pattern, str(x)))
movies['language'] = movies['spoken_languages'].apply(lambda x: extract_concatenate_data(name_pattern, str(x)))

# Normalizar la columna 'cast' en credits
credits['actor'] = credits['cast'].apply(lambda x: extract_concatenate_data(name_pattern, str(x)))

# Normalizar la columna 'crew' en credits
credits['director'] = credits['crew'].apply(lambda x: extract_concatenate_data(director_pattern, str(x)))

Desanidamos los campos `belongs_to_collection, genres, production_companies, production_countries, spoken_languages, cast y crew`

In [8]:
#Desanidar columnas 'belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages'
columns_to_drop_movies = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']
movies = movies.drop(columns=columns_to_drop_movies)

#Eliminar las columnas 'cast', 'crew'
columns_to_drop_credits = ['cast', 'crew']
credits = credits.drop(columns=columns_to_drop_credits)

Mostramos un resumen de los registros nulos y duplicados en cada columna del dataframe.

In [21]:
#crear una funcion para los datos nulos
def evaluate_records(df):
	columns = df.columns
	for column in columns:
		if column != 'id': #Excluimos la columna 'id'
			null_count = df[column].isnull().sum()
			unique_count = len(df[column].unique())
			duplicate_count = df.duplicated(subset=column).sum()
			print(f"The column {column} has {null_count} null records, a total of {unique_count} unique data, and {duplicate_count} duplicate records among them.")

LLenamos los valores nulos de los campos `revenue` y `budget` del dataframe 'movies' con el valor 0

In [10]:
# Llenamos valores nulos con 0
movies['revenue'].fillna(0, inplace=True)
movies['budget'].fillna(0, inplace=True)

In [11]:
# Verificar que no haya valores nulos ('budget', 'revenue')
print("Nulos", movies['budget'].isna().sum())
print("Nulos", movies['revenue'].isna().sum())

Nulos 0
Nulos 0


In [12]:
movies.shape

(45466, 24)

Eliminamos valores nulos del campo `release date`

In [13]:
# Eliminamos valores nulos en release date
movies_cleaned = movies.copy()
movies_cleaned.dropna(subset=['release_date'], inplace=True)

cambiamos a formato fecha `AAAA-mm-dd` y creamos una nueva columna `release_year` con el año de la fecha de estreno.

In [14]:
# Cambio el formato de Fecha
movies['release_date'] = pd.to_datetime(movies['release_date'], format='%Y-%m-%d', errors='coerce')
movies['release_year'] = movies['release_date'].dt.year

In [15]:
# Vemos como queda
movies[['release_date','release_year']].head()

,release_date,release_year
0,1995-10-30,1995.0
1,1995-12-15,1995.0
2,1995-12-22,1995.0
3,1995-12-22,1995.0
4,1995-02-10,1995.0


Llenamos los valores nulos en revenue por el número 0, para luego crear la columna con el retorno de inversión `return=revenue/budget`

In [17]:
# Rellenamos los valores nulos en 'revenue'
movies['revenue'] = movies['revenue'].fillna(0)

# Eliminamos las filas con valores no válidos en 'budget'
movies = movies[pd.to_numeric(movies['budget'], errors='coerce').notnull()]

# Convertimos las columnas 'budget' y 'revenue' a tipo float64
movies['budget'] = movies['budget'].astype(np.float64)
movies['revenue'] = movies['revenue'].astype(np.float64)

# Calculamos el retorno
movies['return'] = movies.apply(lambda x: x['revenue'] / x['budget'] if x['budget'] != 0 else 0, axis=1)

In [22]:
evaluate_records(movies)

The column adult has 0 null records, a total of 2 unique data, and 45461 duplicate records among them.
The column budget has 0 null records, a total of 1223 unique data, and 44240 duplicate records among them.
The column homepage has 37684 null records, a total of 7671 unique data, and 37792 duplicate records among them.
The column imdb_id has 17 null records, a total of 45417 unique data, and 46 duplicate records among them.
The column original_language has 11 null records, a total of 90 unique data, and 45373 duplicate records among them.
The column original_title has 0 null records, a total of 43371 unique data, and 2092 duplicate records among them.
The column overview has 954 null records, a total of 44307 unique data, and 1156 duplicate records among them.
The column popularity has 3 null records, a total of 44176 unique data, and 1287 duplicate records among them.
The column poster_path has 386 null records, a total of 45022 unique data, and 441 duplicate records among them.
The

Eliminar las columnas que no serán utilizadas, `video`,`imdb_id`,`adult`,`original_title`,`poster_path` y `homepage`.

In [23]:
# Eliminar columnas no utilizadas
movies.drop(['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], axis=1, inplace=True)

Despues de las transformaciones y cambiamos el tipo de dato de cada columna en caso de ser necesario

In [24]:
# Cambiamos el tipo de dato de la columna 'popularity'
movies['popularity'] = movies['popularity'].astype(float)

In [25]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45463 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   budget             45463 non-null  float64       
 1   id                 45463 non-null  object        
 2   original_language  45452 non-null  object        
 3   overview           44509 non-null  object        
 4   popularity         45460 non-null  float64       
 5   release_date       45376 non-null  datetime64[ns]
 6   revenue            45463 non-null  float64       
 7   runtime            45203 non-null  float64       
 8   status             45379 non-null  object        
 9   tagline            20412 non-null  object        
 10  title              45460 non-null  object        
 11  vote_average       45460 non-null  float64       
 12  vote_count         45460 non-null  float64       
 13  collection         45463 non-null  object        
 14  genre      

In [26]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        45476 non-null  int64 
 1   actor     45476 non-null  object
 2   director  45476 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [27]:
movies.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,genre,company,country,language,release_year,return
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,"Animation, Comedy, Family",Pixar Animation Studios,United States of America,English,1995.0,12.451801
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,,"Adventure, Fantasy, Family","TriStar Pictures, Teitler Film, Interscope Com...",United States of America,"English, Français",1995.0,4.043035
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"Romance, Comedy","Warner Bros., Lancaster Gate",United States of America,English,1995.0,0.000000
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,,"Comedy, Drama, Romance",Twentieth Century Fox Film Corporation,United States of America,English,1995.0,5.090760
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,Comedy,"Sandollar Productions, Touchstone Pictures",United States of America,English,1995.0,0.000000


In [28]:
credits.head()

,id,actor,director
0,862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,31357,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,11862,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer


#### 1.5 Transformacion de los datos Adicional

Duplicados por cada DataFrame

In [29]:
# Duplicados
print('Duplicados movies:', movies.duplicated().sum())
print('Duplicados credits:', credits.duplicated().sum())

Duplicados movies: 17
Duplicados credits: 43


In [30]:
# Eliminamos los duplicados

movies = movies.drop_duplicates()  
credits = credits.drop_duplicates() 

In [31]:
# Verificamos duplicados
print('ID Duplicados movies:', movies['id'].duplicated().sum())
print('ID Duplicados credits:', credits['id'].duplicated().sum())

ID Duplicados movies: 13
ID Duplicados credits: 1


In [32]:
# Eliminamos registros con 'id' duplicados
movies = movies.drop_duplicates(subset='id')
credits = credits.drop_duplicates(subset='id')

#### Unimos los dos DataFrame

In [33]:
movies['id'] = movies['id'].astype('int64')

In [34]:
moviesCredits = movies.merge(credits, on='id', how='left')

In [35]:
moviesCredits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45433 entries, 0 to 45432
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   budget             45433 non-null  float64       
 1   id                 45433 non-null  int64         
 2   original_language  45422 non-null  object        
 3   overview           44479 non-null  object        
 4   popularity         45430 non-null  float64       
 5   release_date       45346 non-null  datetime64[ns]
 6   revenue            45433 non-null  float64       
 7   runtime            45173 non-null  float64       
 8   status             45349 non-null  object        
 9   tagline            20401 non-null  object        
 10  title              45430 non-null  object        
 11  vote_average       45430 non-null  float64       
 12  vote_count         45430 non-null  float64       
 13  collection         45433 non-null  object        
 14  genre 

Optimizacion de datos

In [36]:
# Seleccionar las columnas deseadas y ordenarlas
moviesCredits = moviesCredits[['id', 'title', 'tagline', 'overview', 'collection', 'genre', 'company',
                               'original_language', 'runtime', 'popularity', 'vote_count', 'vote_average',
                               'release_date', 'release_year', 'status', 'country', 'language',
                               'revenue', 'budget', 'return', 'actor', 'director']]

In [37]:
# Guardar el DataFrame como un archivo CSV
moviesCredits.to_csv('C:/Users/JESHUDELL/Desktop/Proyecto Individual jesu/Dataset/clean_movies.csv', index=False, header=True, sep=';', encoding='utf-8')

# Número de observaciones en el DataFrame
num_observaciones = len(moviesCredits)
print(f"Número de observaciones: {num_observaciones}")


Número de observaciones: 45433


In [38]:
moviesCredits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45433 entries, 0 to 45432
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 45433 non-null  int64         
 1   title              45430 non-null  object        
 2   tagline            20401 non-null  object        
 3   overview           44479 non-null  object        
 4   collection         45433 non-null  object        
 5   genre              45433 non-null  object        
 6   company            45433 non-null  object        
 7   original_language  45422 non-null  object        
 8   runtime            45173 non-null  float64       
 9   popularity         45430 non-null  float64       
 10  vote_count         45430 non-null  float64       
 11  vote_average       45430 non-null  float64       
 12  release_date       45346 non-null  datetime64[ns]
 13  release_year       45346 non-null  float64       
 14  status